 - hash name
 - get python types from function type
 - add a compile function that takes (name, input types, output types) and returns a callable

In [1]:
from pathlib import Path
from xdsl.builder import ImplicitBuilder

from xdsl.dialects import func, arith, memref
from xdsl.dialects.builtin import ModuleOp, i64, f64


def get_module() -> ModuleOp:
    memref_t = memref.MemRefType.from_element_type_and_shape(f64, [100])
    module = ModuleOp([])
    with ImplicitBuilder(module.body):
        f = func.FuncOp("hello", ((i64, i64), (i64,)))
        with ImplicitBuilder(f.body) as (lhs, rhs):
            res = arith.Addi(lhs, rhs).result
            func.Return(res)

        g = func.FuncOp("dist", ((memref_t, memref_t), (f64,)))
        with ImplicitBuilder(g.body) as (lhs, rhs):
            res = arith.Constant.from_float_and_width(1.0, f64)
            func.Return(res)

    return module

str(get_module())

with open(Path() / "input.mlir", 'w') as f:
    f.write(str(get_module()))

In [2]:
from xdsl.jit import jit_module
import ctypes

dbl_ptr_type = ctypes.POINTER(ctypes.c_double)

hey = jit_module(get_module(), "hello", types=((int, int), int))
dist_mlir = jit_module(get_module(), "dist", types=((dbl_ptr_type, dbl_ptr_type), float))
hey(5,6)

11

In [3]:
import numpy as np

In [4]:
n = 100
a = np.random.random(n)
b = np.random.random(n)

In [5]:
a_data_ptr = a.ctypes.data_as(dbl_ptr_type)
b_data_ptr = b.ctypes.data_as(dbl_ptr_type)
dist_mlir(a_data_ptr, b_data_ptr)

(1.0, float)

In [6]:
%time hey(5, 6)

CPU times: user 26 µs, sys: 145 µs, total: 171 µs
Wall time: 19.3 µs


11

In [7]:
%time 1 + 2

CPU times: user 6 µs, sys: 33 µs, total: 39 µs
Wall time: 6.91 µs


3

In [8]:
from typing import Any


def dist_np(lhs: np.ndarray[Any, np.dtype[np.float64]], rhs: np.ndarray[Any, np.dtype[np.float64]]) -> np.float64:
    diff = lhs - rhs
    res = np.linalg.norm(diff)
    return res


In [9]:
%time dist_np(a, b)

CPU times: user 120 µs, sys: 538 µs, total: 658 µs
Wall time: 63.9 µs


4.002462888534158